# Exploratory Analysis of Presto Logs

This is a Jupyter notebook to implement an explorary analysis on Presto request logs.

Before running the machine learning pipeline in the query predictor package, users can have a quick analysis on the logs.

In [ ]:
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## Data loading

In [ ]:
# Load the dataset for analysis.
# The path may be changed for different datasets.
data = pd.read_csv('../datasets/data/presto-logs.csv')
columns = ['query_id', 'user_', 'principal', 'source', 'environment', 'catalog', 'query_state', 
           'query', 'error_code_name', 'failure_type', 'peak_memory_bytes', 'cpu_time_ms', 'datehour']
data = data[columns]
data

In [ ]:
print('starts from:', min(data['datehour']))
print('ends at:', max(data['datehour']))

## Basic analysis on the logs

In [ ]:
principal_df = data[data['principal'] == '-']
principal_df['source'].value_counts()

In [ ]:
env_counts = data['environment'].value_counts()
print(env_counts)
env_counts[:5].plot.pie(y='environment')

In [ ]:
source_counts = data['source'].value_counts()
print(source_counts)
source_counts[:5].plot.pie(y='source')

In [ ]:
cat_counts = data['catalog'].value_counts()
print(cat_counts[:2])

In [ ]:
print(len(data['user_'].unique()))
print(data['user_'].value_counts())

## Analysis of CPU and memory usages

In [ ]:
data['cpu_time_ms'].fillna(0)
data['peak_memory_bytes'].fillna(0)

In [ ]:
def category_cpu(row):
    if row.cpu_time_ms < 1000 * 30: # 30s
        return 0
    elif row.cpu_time_ms < 1000 * 60 * 5: # 5m
        return 1
    elif row.cpu_time_ms < 1000 * 60 * 60: # 1h
        return 2
    elif row.cpu_time_ms < 1000 * 60 * 60 * 5: # 5h
        return 3
    else:
        return 4
    
data['c_label'] = data.apply(category_cpu, axis = 1)
data.hist(column='c_label')

In [ ]:
def category_memory(row):
    if row.peak_memory_bytes < 1024: # 1 MB
        return 0
    elif row.peak_memory_bytes < 1024 * 1024: # 1 GB
        return 1
    elif row.peak_memory_bytes < 1024 * 1024 * 1024: # 1 TB
        return 2
    else:
        return 3

data['m_label'] = data.apply(category_memory, axis = 1)
data.hist(column='m_label')

## Analysis of failed queries

In [ ]:
state_counts = data['query_state'].value_counts()
print(state_counts)
total_count = state_counts[0] + state_counts[1]
print('Success:', state_counts[0] / total_count)
print('Failure:', state_counts[1] / total_count)

In [ ]:
data['error_code_name'].value_counts()[:10]